# Practical 11: American option pricing with finite differences

Computational Finance with Python

[Alet Roux](https://www.york.ac.uk/maths/staff/alet-roux/) ([Department
of Mathematics](https://maths.york.ac.uk), University of York)

Click on the following to open this file in Google Colab:

<figure>
<a
href="https://colab.research.google.com/github/aletroux/comp-finance-python/blob/main/practicals/P11_finite_difference_American_options_prac.ipynb"><img
src="https://colab.research.google.com/assets/colab-badge.svg"
alt="Open In Colab" /></a>
<figcaption>Open In Colab</figcaption>
</figure>

The aim of this practical is to use finite difference methods to price
American put options.

# 1. European option

The code in the following cell performs a finite difference
approximation to the price of a European put option with strike $K=90$
and maturity date $T=1$ using the Crank-Nicolson method in the
Black-Scholes model with parameters $S_0=100$, $r=0.02$ and
$\sigma=0.2$. In this case the partial differential equation and initial
value is $$\begin{aligned}
\frac{\partial V}{\partial \tau}(\tau,S) &= r S\frac{\partial V}{\partial S}(\tau,S) + \tfrac{1}{2}\sigma^2 S^2\frac{\partial^2 V}{\partial S^2}(\tau,S)  -r V(\tau,S), \\
V(0, S) &= (K-S)^+ \text{ for all }S\in[0,\infty).
\end{aligned}
$$ We use the boundary conditions $$\begin{aligned}
\lim_{S\rightarrow 0} \frac{\partial^2 V}{\partial S^2}(\tau,S) &= 0, \\
\lim_{S\rightarrow \infty} V(\tau, S) &= 0.
\end{aligned}
$$

The structure of the code below should be familiar from previous
practicals. Run it and check the results.

In [1]:
import numpy as np
import math
import scipy.sparse as ssparse

def CN_price_put (Smin, Smax, T, K, r, sigma, M, N):    
    """Finite difference approximation of put option price in Black-Scholes model
    using Crank-Nicolson method.
    Arguments:
        Smin: lower boundary of grid
        Smax: upper boundary of grid
        T: maturity date
        K: strike
        r: interest rate
        r: constant volatility
        M: number of steps in direction tau
        N: number of steps in direction S
    Returns:
        V: matrix of results, V[i,j] is value at 
            iT/M and Smin + j(Smax-Smin)/N
        tau: vector of time steps
        S: vector of stock prices
    """

    V = np.zeros((M+1,N+1))

    # discretization parameters
    tau = np.linspace(0, T, M+1)
    S = np.linspace(0, Smax, N+1)
    Sint = S[1:-1]    
    dtau = tau[1]-tau[0]
    dS = S[1] - S[0]
    
    #initial value
    V[0] = np.maximum(K - S,0)

    # parameters that don't depend on time step
    alpha = dtau/2/dS**2 * sigma**2 * Sint**2
    beta = r*dtau/2/dS*Sint

    # Backward Euler matrix and vector
    uB = - alpha - beta
    dB = 2*alpha + r*dtau + 1
    lB = beta - alpha
    #impose boundary conditions
    uB[0] -= lB[0]
    dB[0] += 2*lB[0]
    #matrix
    AB = ssparse.diags(diagonals=[1+dB, lB[1:], uB[:-1]], offsets=[0, -1, 1], 
                       shape=(N-1, N-1), format='csc')
    ABsolver = ssparse.linalg.factorized(AB)

    # Forward Euler matrix and vector
    uF = alpha + beta
    dF = 1 - r*dtau- 2*alpha
    lF = alpha - beta
    #impose boundary conditions
    uF[0] -= lF[0]
    dF[0] += 2*lF[0]
    #matrix
    AF = ssparse.diags(diagonals=[1+dF, lF[1:], uF[:-1]], offsets=[0, -1, 1], 
                       shape=(N-1, N-1), format='csc')

    for i in range(M):
        V[i+1, 1:-1] = ABsolver(AF @ V[i, 1:-1])

        #impose boundary conditions
        V[i+1,0] = -V[i+1,2] + 2*V[i+1,1]

    return V, tau, S
                    
# Strike
K = 90

# Black-Scholes parameters
T = 1
S0 = 100
r = 0.05
sigma = 0.2

# FD parameters
Smin = 0
Smax = 2*S0

M = 200
N = 200

VE, tau, S = CN_price_put (Smin, Smax, T, K, r, sigma, M, N)


In [2]:
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

fig = plt.figure(figsize = (8,6), constrained_layout=True)
ax = plt.axes(projection='3d')
ax.view_init(elev=20, azim=135)

ax.set_xlabel("Time to maturity $\\tau$")
ax.set(xlim = [tau[0],tau[-1]])
ax.tick_params(axis='x', pad=0)
ax.xaxis.pane.fill = False

ax.set_ylabel("Stock price $S$")              
ax.set(ylim = [S[0],S[-1]])
ax.tick_params(axis='y', pad=0)
ax.yaxis.pane.fill = False

ax.set_zlabel("$V(\tau,S)$")              
ax.zaxis.pane.fill = False

Ss, taus = np.meshgrid(S, tau)

ax.set(title = f"Crank-Nicolson approximation for European put option with strike $K={K}$")

surf = ax.plot_surface(taus, Ss, VE, rstride=1, cstride=1, linewidth=0, cmap='viridis')
fig.colorbar(surf, shrink=0.5, aspect=5);

In [3]:
fig, ax = plt.subplots(figsize = (12,7))

# approximation
ax.plot(S, VE[-1], marker = '.')

# plot cosmetics
ax.set(title = f"Crank-Nicolson approximation at time to maturity $\\tau=T$ for European put option "
       f"with strike $K={K}$")
ax.set(xlabel="Stock price $S$", xlim=(Smin, Smax))
ax.xaxis.grid(True)
ax.yaxis.grid(True)

# 2. American put option pricing

<span class="theorem-title">**Exercise 1**</span> Modify the code above
to compute the price of an American put option by using the
Brennan-Schwartz algorithm.

Code for solving a tridiagonal system is provided below.

Compare your results with the results of the European put option
calculated above.

In [4]:
def tri_matrix_solver_sparse (A, b):
    """Solves tridiagonal system Ax = b. A is in sparse diagonal format with A.offsets == [1, 0, -1]"""

    if not (A.offsets == np.array([1,0,-1])).all():
        print("Diagonal offsets of A in incorrect order: should be 1,0,-1.")
        return None

    u = A.data[0]
    d = A.data[1]
    l = A.data[2]
    
    ddash = np.array(d, dtype=float)
    bdash = np.array(b, dtype=float)

    n = len(d)

    # step 2
    for k in range(1,n):
        ddash[k] -= l[k-1]/ddash[k-1]*u[k]
        bdash[k] -= l[k-1]/ddash[k-1]*bdash[k-1]
        
    x = np.empty_like(b, dtype=float)

    #step 3
    x[-1] = bdash[-1]/ddash[-1]

    #step 4
    for k in range(n-2,-1,-1):
        x[k] = (bdash[k] - u[k+1] * x[k+1])/ddash[k]

    return x

# 3. Greeks

<span class="theorem-title">**Exercise 2**</span> Approximate the value
of the gamma and delta for the American put option at all the grid
points used for the finite difference approximation.

Present your results graphically.

In [9]:
#Insert code here

# 4. Exercise boundary

The **exercise boundary** of the American put option is a curve
$\tau\mapsto S^\ast(\tau)$ that if $S \le S^\ast(\tau)$, then the value
$V(\tau,S)$ of the American put option is at time to maturity $\tau$ is
equal to its payoff $K - S$ if possible (and it is optimal to exercise),
while for $S > S^\ast(t)$ the value exceeds this immediate exercise
value (and it is optimal to continue holding the option). It can be
approximated using the grid $V_{i,j}$ ($i=0, \ldots, M$, $j=0,\ldots,N$)
of approximated values obtained by finite differences, by setting
$S^\ast_i=S_{j^\ast_i}$ for each $i=0, \ldots, M$ in such a way that
$j^\ast_i$ is the largest possible value of $j$ such that
$$V_{i,j} = \max\{K-S_j,0\}.$$

<span class="theorem-title">**Exercise 3**</span> Use the numerical
results produced so far to approximate the exercise boundary of the
American put option.